In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
!export PYSPARK_PYTHON=python3.6.9

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout,Flatten
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import subprocess
import tensorflow as tf
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.corpus import brown
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit
import pickle
import tensorflow.keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json
from tensorflow import keras
from pyspark.sql.functions import udf
import pandas as pd

spark.sparkContext.getConf().getAll()

Using TensorFlow backend.


[('spark.eventLog.enabled', 'true'),
 ('spark.executor.instances', '3'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.cores', '5'),
 ('spark.executor.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.host', 'kafka1'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.yarn.archive', 'hdfs:///user/spark/conf/spark-libs.jar'),
 ('spark.driver.port', '33401'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.shuffle.partitions', '200'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.kryoserializer.buffer.max', '2047'),
 ('spark.app.id', 'application_1667038354005_0002'),
 ('spark.executor.memoryOverhead', '1g'),
 ('spark.driver.memoryOverhead', '1g'),
 ('spark.executor.memory', '4g'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.

In [3]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout,Flatten
from keras import backend as K
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import subprocess
import nltk

In [5]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [6]:
spark

In [7]:
sc = spark._jsc.sc() 

In [8]:
sc

JavaObject id=o165

In [9]:
print(sc.version)

In [10]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json

In [11]:
spark 

In [12]:
sc = spark.sparkContext

In [13]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [14]:
sql_context = SQLContext(sc)

In [15]:
sql_context

In [16]:
sc.master

'yarn'

# 데이터프레임

In [17]:
import re

def regex_(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['total_text_list'] = text['total_text_list'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['total_text_list'] = text['total_text_list'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['total_text_list'] = text['total_text_list'].str.replace(pat=pattern,repl=r'',regex=True)
    text['total_text_list'] = text['total_text_list'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['total_text_list'] = text['total_text_list'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    text['total_text_list'] = text['total_text_list'].str.lower()
    text=text[((text['total_text_list'].str.len()>= 10)) & ((text['total_text_list'].eq(' ')==False) & (text['total_text_list'].eq('')==False))]
    return text
def regex2_(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['text'] = text['text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['text'] = text['text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['text'] = text['text'].str.replace(pat=pattern,repl=r'',regex=True)
    text['text'] = text['text'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['text'] = text['text'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    text['text'] = text['text'].str.lower()
    text=text[((text['text'].str.len()>= 10)) & ((text['text'].eq(' ')==False) & (text['text'].eq('')==False))]
    return text

def regex_3(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    # @로 시작하는 애들도 정규식 이용해서 없애기 @[^ ]+
    text['full_text'] = text['full_text'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.lower()
    text=text[((text['full_text'].str.len()>= 10)) & ((text['full_text'].str.split(" ").str.len()>= 10)) & ((text['full_text'].eq(' ')==False) & (text['full_text'].eq('')==False))]
    return text

def regex_4(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    # @로 시작하는 애들도 정규식 이용해서 없애기 @[^ ]+
    text['full_text'] = text['full_text'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.lower()
    #text=text[((text['full_text'].str.len()>= 10)) & ((text['full_text'].str.split(" ").str.len()>= 10)) & ((text['full_text'].eq(' ')==False) & (text['full_text'].eq('')==False))]
    return text

# general_minsun2.csv
general = pd.read_csv('/root/spark/bigcomp_model/general.csv', sep=',',  lineterminator='\n')
general = general.drop_duplicates()
general = spark.createDataFrame(general[['total_text_list']])
general = general.select(col('total_text_list').alias('text')) 
general = general.withColumn('label', lit(0))
general.show(5)
print(general.count())
general = general.withColumn("label",col("label").cast("integer"))


/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  An error occurred while calling z:org.apache.spark.sql.api.python.PythonSQLUtils.readArrowStreamFromFile.
: java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.readNextBatch(ArrowConverters.scala:243)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.<init>(ArrowConverters.scala:229)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$.getBatchesFromStream(ArrowConverters.scala:228)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun$readArrowStreamFromFile$2.apply(ArrowConverters.scala:216)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun

+--------------------+-----+
|                text|label|
+--------------------+-----+
|im at ap stgate i...|    0|
|why do i love thi...|    0|
|nowplaying charmi...|    0|
|im at kuwait city...|    0|
|lolim right hande...|    0|
+--------------------+-----+
only showing top 5 rows

801932


In [18]:
# 보안 관련 계정들 트윗
security = pd.read_csv('/root/spark/bigcomp_model/dataset.csv', sep=',',  lineterminator='\n')
security = security[["text","datetime"]]
security = security.sort_values(by='datetime' ,ascending=True)
security = security.reset_index(drop=True)
security = regex2_(security)
security = security[~security['text'].str.contains("rt", na=False, case=False)]
security = security.drop_duplicates()


In [19]:
security = spark.createDataFrame(security[['text']])
security = security.withColumn('label', lit(1))
security = security.withColumn("label",col("label").cast("integer"))

/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  An error occurred while calling z:org.apache.spark.sql.api.python.PythonSQLUtils.readArrowStreamFromFile.
: java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.readNextBatch(ArrowConverters.scala:243)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.<init>(ArrowConverters.scala:229)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$.getBatchesFromStream(ArrowConverters.scala:228)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun$readArrowStreamFromFile$2.apply(ArrowConverters.scala:216)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun

In [20]:
pretrained1 = security.collect()[:200000]
pretrained2 = general.collect()[:200000]


pretrained1 = spark.createDataFrame(pretrained1)
pretrained2 = spark.createDataFrame(pretrained2)
pre = pretrained1.union(pretrained2)

In [21]:
t1 = security.collect()[200000:250000]
t2 = security.collect()[250000:300000]
t3 = security.collect()[300000:350000]
t4 = security.collect()[350000:400000]

t5 = general.collect()[200000:250000]
t6 = general.collect()[250000:300000]
t7 = general.collect()[300000:350000]
t8 = general.collect()[350000:400000]


t1 = spark.createDataFrame(t1)
t2 = spark.createDataFrame(t2)
t3 = spark.createDataFrame(t3)
t4 = spark.createDataFrame(t4)
t5 = spark.createDataFrame(t5)
t6 = spark.createDataFrame(t6)
t7 = spark.createDataFrame(t7)
t8 = spark.createDataFrame(t8)

df1 = t1.union(t5)
df2 = t2.union(t6)
df3 = t3.union(t7)
df4 = t4.union(t8)

"""word1 = security.collect()[200000:250000]
word2 = general.collect()[200000:250000]

word3 = security.collect()[200000:300000]
word4 = general.collect()[200000:300000]

word5 = security.collect()[200000:350000]
word6 = general.collect()[200000:350000]

word7 = security.collect()[200000:400000]
word8 = general.collect()[200000:400000]"""


'word1 = security.collect()[200000:250000]\nword2 = general.collect()[200000:250000]\n\nword3 = security.collect()[200000:300000]\nword4 = general.collect()[200000:300000]\n\nword5 = security.collect()[200000:350000]\nword6 = general.collect()[200000:350000]\n\nword7 = security.collect()[200000:400000]\nword8 = general.collect()[200000:400000]'

In [22]:
word1 = security.collect()[:250000]
word2 = general.collect()[:250000]

word3 = security.collect()[:300000]
word4 = general.collect()[:300000]

word5 = security.collect()[:350000]
word6 = general.collect()[:350000]

word7 = security.collect()[:400000]
word8 = general.collect()[:400000]

In [23]:
word1 = spark.createDataFrame(word1)
word2= spark.createDataFrame(word2)

word3 = spark.createDataFrame(word3)
word4= spark.createDataFrame(word4)

word5 = spark.createDataFrame(word5)
word6= spark.createDataFrame(word6)

word7 = spark.createDataFrame(word7)
word8= spark.createDataFrame(word8)

In [22]:
first = t1.union(t5)
second = t1.union(t2).union(t5).union(t6)
third = t1.union(t2).union(t3).union(t5).union(t6).union(t7)
fourth = t1.union(t2).union(t3).union(t4).union(t5).union(t6).union(t7).union(t8)

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
def background_keyword(df): # input : spark dataframe을 pandas로 변환한 후, dataframe column
    background = df.values.tolist()
    token = Tokenizer()         # 토큰화 함수 지정
    token.fit_on_texts(background)    # 토큰화 함수에 문장 적용
    sorted_dic4 = sorted(token.word_counts.items(), key=lambda x: x[1], reverse=True)
    word_counts_1 = {}
    for i in sorted_dic4:
        word_counts_1.update({i[0]:i[1]})
    return word_counts_1 # dictionary 임
def back_keyword(pos,neg): # pos : event-related keyword, neg : event-unrelated keyword
    stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
    stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
    word1 = stage_1.transform(pos) #나는 first_train 부터 함!
    word1 = stage_2.transform(word1)
    word2 = stage_1.transform(neg) #나는 first_train 부터 함!
    word2 = stage_2.transform(word2)
    from pyspark.sql.functions import col, concat_ws
    df1 = word1.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word1 = df1.select("filtered_words").toPandas()
    df2 = word2.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word2 = df2.select("filtered_words").toPandas()
    word_counts_1 = background_keyword(word1["filtered_words"])
    word_counts_2 = background_keyword(word2["filtered_words"])
    return word_counts_1, word_counts_2




# Author : Minseon Kim (2021)
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = sc.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

# 바꾼 버전
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame
from pyspark.sql import Row
from pyspark.sql import types   

def series_to_list(x):
    seri = x.values.tolist()
    for i in range(len(seri)):
        seri[i] = seri[i].tolist()
    return seri
class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer(lower=False)
    neg_t = Tokenizer(lower=False)
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        result_pdf = df.select("filtered_words").toPandas()
        pos_df = result_pdf[:200000] # 24939 99756
        neg_df = result_pdf[200000:]
        print(len(result_pdf))
        
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        print(www1['allow'])
        print(www2['like'])
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        result_list = series_to_list(result_pdf["filtered_words"])
        encoded_docs_pos = self.pos_t.texts_to_sequences(result_list) # 상위 event keyword 5k가 쓰인 tokenizer
        encoded_docs_neg = self.neg_t.texts_to_sequences(result_list)
        
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post').tolist()
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post').tolist()
        
        text_array = [str(row['text']) for row in df.select('text').collect()]
        label_array = [int(row['label']) for row in df.select('label').collect()]

        
        zip_array = list(zip(text_array, label_array, X_p, X_n))
        rdd = sc.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature1','feature2'])
        

        print(type(fdf))

        return fdf, www1, aa
    
    
word_counts_1, word_counts_2 = back_keyword(word1, word2)
word_counts_1 = sc.broadcast(word_counts_1)
word_counts_2 = sc.broadcast(word_counts_2)

stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

r1 = stage_1.transform(first)
r2 = stage_2.transform(r1)


# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
df_example, www2, bb = text_sequence.transform(r2)
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["label"], 
    list_to_vector_udf(df_example["feature1"]).alias("feature1"), 
    list_to_vector_udf(df_example["feature2"]).alias("feature2")
)
label_str_index = StringIndexer(inputCol='label', outputCol='label_index') # transformer의 마지막 단계!!
pre_df = label_str_index.fit(df_with_vectors).transform(df_with_vectors)


AttributeError: 'function' object has no attribute '_jdf'

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
def background_keyword(df): # input : spark dataframe을 pandas로 변환한 후, dataframe column
    background = df.values.tolist()
    token = Tokenizer()         # 토큰화 함수 지정
    token.fit_on_texts(background)    # 토큰화 함수에 문장 적용
    sorted_dic4 = sorted(token.word_counts.items(), key=lambda x: x[1], reverse=True)
    word_counts_1 = {}
    for i in sorted_dic4:
        word_counts_1.update({i[0]:i[1]})
    return word_counts_1 # dictionary 임
def back_keyword(pos,neg): # pos : event-related keyword, neg : event-unrelated keyword
    stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
    stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
    word1 = stage_1.transform(pos) #나는 first_train 부터 함!
    word1 = stage_2.transform(word1)
    word2 = stage_1.transform(neg) #나는 first_train 부터 함!
    word2 = stage_2.transform(word2)
    from pyspark.sql.functions import col, concat_ws
    df1 = word1.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word1 = df1.select("filtered_words").toPandas()
    df2 = word2.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word2 = df2.select("filtered_words").toPandas()
    word_counts_1 = background_keyword(word1["filtered_words"])
    word_counts_2 = background_keyword(word2["filtered_words"])
    return word_counts_1, word_counts_2




# Author : Minseon Kim (2021)
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = sc.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

# 바꾼 버전
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame
from pyspark.sql import Row
from pyspark.sql import types   

def series_to_list(x):
    seri = x.values.tolist()
    for i in range(len(seri)):
        seri[i] = seri[i].tolist()
    return seri
class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer(lower=False)
    neg_t = Tokenizer(lower=False)
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        result_pdf = df.select("filtered_words").toPandas()
        pos_df = result_pdf[:200000] # 24939 99756
        neg_df = result_pdf[200000:]
        print(len(result_pdf))
        
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        print(www1['allow'])
        print(www2['like'])
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        result_list = series_to_list(result_pdf["filtered_words"])
        encoded_docs_pos = self.pos_t.texts_to_sequences(result_list) # 상위 event keyword 5k가 쓰인 tokenizer
        encoded_docs_neg = self.neg_t.texts_to_sequences(result_list)
        
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post').tolist()
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post').tolist()
        
        text_array = [str(row['text']) for row in df.select('text').collect()]
        label_array = [int(row['label']) for row in df.select('label').collect()]
        X = np.array(X_p)
        y = np.array(label_array)

        
        zip_array = list(zip(text_array, label_array, X_p, X_n))
        rdd = sc.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature1','feature2'])
        

        print(type(fdf))

        return fdf, www1, aa, X, y
    
    
word_counts_1, word_counts_2 = back_keyword(pretrained1, pretrained2)
word_counts_1 = sc.broadcast(word_counts_1)
word_counts_2 = sc.broadcast(word_counts_2)

stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

r1 = stage_1.transform(pre)
r2 = stage_2.transform(r1)


# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
df_example, www2, bb, X, y = text_sequence.transform(r2)
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["label"], 
    list_to_vector_udf(df_example["feature1"]).alias("feature1"), 
    list_to_vector_udf(df_example["feature2"]).alias("feature2")
)
label_str_index = StringIndexer(inputCol='label', outputCol='label_index') # transformer의 마지막 단계!!
pre_df = label_str_index.fit(df_with_vectors).transform(df_with_vectors)


400000
1270
29356
<class 'pyspark.sql.dataframe.DataFrame'>


In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
def background_keyword(df): # input : spark dataframe을 pandas로 변환한 후, dataframe column
    background = df.values.tolist()
    token = Tokenizer()         # 토큰화 함수 지정
    token.fit_on_texts(background)    # 토큰화 함수에 문장 적용
    sorted_dic4 = sorted(token.word_counts.items(), key=lambda x: x[1], reverse=True)
    word_counts_1 = {}
    for i in sorted_dic4:
        word_counts_1.update({i[0]:i[1]})
    return word_counts_1 # dictionary 임
def back_keyword(pos,neg): # pos : event-related keyword, neg : event-unrelated keyword
    stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
    stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
    word1 = stage_1.transform(pos) #나는 first_train 부터 함!
    word1 = stage_2.transform(word1)
    word2 = stage_1.transform(neg) #나는 first_train 부터 함!
    word2 = stage_2.transform(word2)
    from pyspark.sql.functions import col, concat_ws
    df1 = word1.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word1 = df1.select("filtered_words").toPandas()
    df2 = word2.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word2 = df2.select("filtered_words").toPandas()
    word_counts_1 = background_keyword(word1["filtered_words"])
    word_counts_2 = background_keyword(word2["filtered_words"])
    return word_counts_1, word_counts_2




# Author : Minseon Kim (2021)
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = sc.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

# 바꾼 버전
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame
from pyspark.sql import Row
from pyspark.sql import types   

def series_to_list(x):
    seri = x.values.tolist()
    for i in range(len(seri)):
        seri[i] = seri[i].tolist()
    return seri
class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer(lower=False)
    neg_t = Tokenizer(lower=False)
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        result_pdf = df.select("filtered_words").toPandas()
        pos_df = result_pdf[:100000] # 24939 99756
        neg_df = result_pdf[100000:]
        print(len(result_pdf))
        
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        print(www1['allow'])
        print(www2['like'])
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        result_list = series_to_list(result_pdf["filtered_words"])
        encoded_docs_pos = self.pos_t.texts_to_sequences(result_list) # 상위 event keyword 5k가 쓰인 tokenizer
        encoded_docs_neg = self.neg_t.texts_to_sequences(result_list)
        
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post').tolist()
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post').tolist()
        
        text_array = [str(row['text']) for row in df.select('text').collect()]
        label_array = [int(row['label']) for row in df.select('label').collect()]

        
        zip_array = list(zip(text_array, label_array, X_p, X_n))
        rdd = sc.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature1','feature2'])
        

        print(type(fdf))

        return fdf, www1, aa
    
    
word_counts_1, word_counts_2 = back_keyword(word3, word4)
word_counts_1 = sc.broadcast(word_counts_1)
word_counts_2 = sc.broadcast(word_counts_2)

stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

r1 = stage_1.transform(second)
r2 = stage_2.transform(r1)


# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
df_example, www2, bb = text_sequence.transform(r2)
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["label"], 
    list_to_vector_udf(df_example["feature1"]).alias("feature1"), 
    list_to_vector_udf(df_example["feature2"]).alias("feature2")
)
label_str_index = StringIndexer(inputCol='label', outputCol='label_index') # transformer의 마지막 단계!!
label_df1 = label_str_index.fit(df_with_vectors).transform(df_with_vectors)


200000
1247
31544
<class 'pyspark.sql.dataframe.DataFrame'>


In [18]:


t1 = security.collect()[400000:450000]
t2 = security.collect()[450000:500000]

t3 = general.collect()[400000:450000]
t4 = general.collect()[450000:500000]

t1 = spark.createDataFrame(t1)
t2 = spark.createDataFrame(t2)
t3 = spark.createDataFrame(t3)
t4 = spark.createDataFrame(t4)


fine1 = t1.union(t3)
fine2 = t2.union(t4)

df1 = t1.union(t3)
df2 = t1.union(t2).union(t3).union(t4)

word1 = security.collect()[200000:450000]
word2 = general.collect()[200000:450000]

word3 = security.collect()[200000:500000]
word4 = general.collect()[200000:500000]


word1 = spark.createDataFrame(word1)
word2= spark.createDataFrame(word2)

word3 = spark.createDataFrame(word3)
word4= spark.createDataFrame(word4)


In [19]:
word1 = security.collect()[200000:450000]
word2 = general.collect()[200000:450000]

word3 = security.collect()[200000:500000]
word4 = general.collect()[200000:500000]


word1 = spark.createDataFrame(word1)
word2= spark.createDataFrame(word2)

word3 = spark.createDataFrame(word3)
word4= spark.createDataFrame(word4)
dff1 = word1.union(word2)
dff2 = word3.union(word4)


In [20]:
dff2 = word3.union(word4)

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
def background_keyword(df): # input : spark dataframe을 pandas로 변환한 후, dataframe column
    background = df.values.tolist()
    token = Tokenizer()         # 토큰화 함수 지정
    token.fit_on_texts(background)    # 토큰화 함수에 문장 적용
    sorted_dic4 = sorted(token.word_counts.items(), key=lambda x: x[1], reverse=True)
    word_counts_1 = {}
    for i in sorted_dic4:
        word_counts_1.update({i[0]:i[1]})
    return word_counts_1 # dictionary 임
def back_keyword(pos,neg): # pos : event-related keyword, neg : event-unrelated keyword
    stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
    stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
    word1 = stage_1.transform(pos) #나는 first_train 부터 함!
    word1 = stage_2.transform(word1)
    word2 = stage_1.transform(neg) #나는 first_train 부터 함!
    word2 = stage_2.transform(word2)
    from pyspark.sql.functions import col, concat_ws
    df1 = word1.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word1 = df1.select("filtered_words").toPandas()
    df2 = word2.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word2 = df2.select("filtered_words").toPandas()
    word_counts_1 = background_keyword(word1["filtered_words"])
    word_counts_2 = background_keyword(word2["filtered_words"])
    return word_counts_1, word_counts_2




# Author : Minseon Kim (2021)
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = sc.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

# 바꾼 버전
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame
from pyspark.sql import Row
from pyspark.sql import types   

def series_to_list(x):
    seri = x.values.tolist()
    for i in range(len(seri)):
        seri[i] = seri[i].tolist()
    return seri
class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer(lower=False)
    neg_t = Tokenizer(lower=False)
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        result_pdf = df.select("filtered_words").toPandas()
        pos_df = result_pdf[:250000] # 24939 99756
        neg_df = result_pdf[250000:]
        print(len(result_pdf))
        
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        print(www1['allow'])
        print(www2['like'])
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        result_list = series_to_list(result_pdf["filtered_words"])
        encoded_docs_pos = self.pos_t.texts_to_sequences(result_list) # 상위 event keyword 5k가 쓰인 tokenizer
        encoded_docs_neg = self.neg_t.texts_to_sequences(result_list)
        
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post').tolist()
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post').tolist()
        
        text_array = [str(row['text']) for row in df.select('text').collect()]
        label_array = [int(row['label']) for row in df.select('label').collect()]

        
        zip_array = list(zip(text_array, label_array, X_p, X_n))
        rdd = sc.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature1','feature2'])
        

        print(type(fdf))

        return fdf, www1, aa
    
    
word_counts_1, word_counts_2 = back_keyword(word1, word2)
word_counts_1 = sc.broadcast(word_counts_1)
word_counts_2 = sc.broadcast(word_counts_2)

stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

r1 = stage_1.transform(dff1)
r2 = stage_2.transform(r1)


# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
df_example, www2, bb = text_sequence.transform(r2)
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["label"], 
    list_to_vector_udf(df_example["feature1"]).alias("feature1"), 
    list_to_vector_udf(df_example["feature2"]).alias("feature2")
)
label_str_index = StringIndexer(inputCol='label', outputCol='label_index') # transformer의 마지막 단계!!
label_df = label_str_index.fit(df_with_vectors).transform(df_with_vectors)


500000
1438
43620
<class 'pyspark.sql.dataframe.DataFrame'>


In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
def background_keyword(df): # input : spark dataframe을 pandas로 변환한 후, dataframe column
    background = df.values.tolist()
    token = Tokenizer()         # 토큰화 함수 지정
    token.fit_on_texts(background)    # 토큰화 함수에 문장 적용
    sorted_dic4 = sorted(token.word_counts.items(), key=lambda x: x[1], reverse=True)
    word_counts_1 = {}
    for i in sorted_dic4:
        word_counts_1.update({i[0]:i[1]})
    return word_counts_1 # dictionary 임
def back_keyword(pos,neg): # pos : event-related keyword, neg : event-unrelated keyword
    stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
    stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
    word1 = stage_1.transform(pos) #나는 first_train 부터 함!
    word1 = stage_2.transform(word1)
    word2 = stage_1.transform(neg) #나는 first_train 부터 함!
    word2 = stage_2.transform(word2)
    from pyspark.sql.functions import col, concat_ws
    df1 = word1.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word1 = df1.select("filtered_words").toPandas()
    df2 = word2.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word2 = df2.select("filtered_words").toPandas()
    word_counts_1 = background_keyword(word1["filtered_words"])
    word_counts_2 = background_keyword(word2["filtered_words"])
    return word_counts_1, word_counts_2




# Author : Minseon Kim (2021)
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = sc.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

# 바꾼 버전
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame
from pyspark.sql import Row
from pyspark.sql import types   

def series_to_list(x):
    seri = x.values.tolist()
    for i in range(len(seri)):
        seri[i] = seri[i].tolist()
    return seri
class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer(lower=False)
    neg_t = Tokenizer(lower=False)
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        result_pdf = df.select("filtered_words").toPandas()
        pos_df = result_pdf[:300000] # 24939 99756
        neg_df = result_pdf[300000:]
        print(len(result_pdf))
        
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        print(www1['allow'])
        print(www2['like'])
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        result_list = series_to_list(result_pdf["filtered_words"])
        encoded_docs_pos = self.pos_t.texts_to_sequences(result_list) # 상위 event keyword 5k가 쓰인 tokenizer
        encoded_docs_neg = self.neg_t.texts_to_sequences(result_list)
        
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post').tolist()
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post').tolist()
        
        text_array = [str(row['text']) for row in df.select('text').collect()]
        label_array = [int(row['label']) for row in df.select('label').collect()]

        
        zip_array = list(zip(text_array, label_array, X_p, X_n))
        rdd = sc.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature1','feature2'])
        

        print(type(fdf))

        return fdf, www1, aa
    
    
word_counts_1, word_counts_2 = back_keyword(word3, word4)
word_counts_1 = sc.broadcast(word_counts_1)
word_counts_2 = sc.broadcast(word_counts_2)

stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

r1 = stage_1.transform(dff2) #나는 first_train 부터 함!
r2 = stage_2.transform(r1)





In [22]:
# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
df_example, www2, bb = text_sequence.transform(r2)
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["label"], 
    list_to_vector_udf(df_example["feature1"]).alias("feature1"), 
    list_to_vector_udf(df_example["feature2"]).alias("feature2")
)
label_str_index = StringIndexer(inputCol='label', outputCol='label_index') # transformer의 마지막 단계!!
label_df1 = label_str_index.fit(df_with_vectors).transform(df_with_vectors)

600000
1524
52344
<class 'pyspark.sql.dataframe.DataFrame'>


In [24]:
spark

In [22]:
label_df.show(2)

+-----+--------------------+--------------------+-----------+
|label|            feature1|            feature2|label_index|
+-----+--------------------+--------------------+-----------+
|    1|[84.0,6.0,13.0,19...|[3097.0,659.0,340...|        1.0|
|    1|[3425.0,2844.0,15...|[1110.0,0.0,0.0,0...|        1.0|
+-----+--------------------+--------------------+-----------+
only showing top 2 rows



In [ ]:
label_df1.show(5)

In [26]:
label_df.show(2)

+-----+--------------------+--------------------+-----------+
|label|            feature1|            feature2|label_index|
+-----+--------------------+--------------------+-----------+
|    1|[96.0,8.0,11.0,22...|[3289.0,662.0,341...|        1.0|
|    1|[4419.0,2972.0,16...|[1130.0,0.0,0.0,0...|        1.0|
+-----+--------------------+--------------------+-----------+
only showing top 2 rows



In [28]:
pretr, prets = pre_df.randomSplit(weights=[0.8,0.2])

In [23]:
trainqq, testqq = label_df.randomSplit(weights=[0.8,0.2])

In [29]:
trainqq1, testqq1 = label_df1.randomSplit(weights=[0.8,0.2])

In [28]:
trainqq.toPandas().to_csv("/root/spark/bigcomp_model/security/fifth_train.csv",index=False)

/root/spark/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [29]:
testqq.toPandas().to_csv("/root/spark/bigcomp_model/security/fifth_test.csv",index=False)

In [24]:
trainqq1.toPandas().to_csv("/root/spark/bigcomp_model/security/sixth_train.csv",index=False)

/root/spark/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [25]:
testqq1.toPandas().to_csv("/root/spark/bigcomp_model/security/six_test.csv",index=False)

In [30]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM, SimpleRNN
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D
nb_classes=2
vocab_sizes = 5001
neg_vocab_sizes = 5001
embedding_vector_length = 100
hidden_dims = 250
max_length = 100
dropout_ratio = 0.5
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_sizes, embedding_vector_length, input_length=max_length))

#어휘 크기(최대 정수 인덱스+1, 고밀도 임베딩의 치수, 입력 시퀀스의 길이

# 모델은 크기의 정수 행렬 (배치,
# input_length) 및 입력에서 가장 큰 정수 (즉, 단어 인덱스)
#은 999 (어휘 크기)보다 크지 않아야합니다.
# 이제 model.output_shape는 (None, 10, 64)이고, 여기서`None`은 배치입니다.
# dimension.
"""model.add(Conv1D(128, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(250, activation='relu'))"""

"""model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))"""

"""#model.add(Activation('relu'))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(256, 3,  padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
#model.add(Dense(128, activation='relu')) #
model.add(Dropout(dropout_ratio))
model.add(Activation('relu'))
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])"""

model.add(Bidirectional(LSTM(128)))
# model.add(GlobalMaxPooling1D())
model.add(Activation('relu')) # 한 층 더 쌓을까..? relu 함수는 정류된 함수로 은닉층에서 많이 사용된다. -를 차단함!
# 은닉층으로 많이 사용되는 이유는 음수는 0으로 반환하므로 특정 양수값으로 수렴하지 않음. 그래서 기울기 소실이 발생하지 않음!
# 시그모이드 함수는 기울기 소실이 발생한다는 단점이 존재. 그래서 relu를 은닉층으로 쓰는 거임!
# 또한, 학습 속도가 매우 빠르다.
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

"""model.add(SimpleRNN(128,return_sequences=True))
model.add(SimpleRNN(128,return_sequences=True))
model.add(SimpleRNN(128))
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])"""

# #체크포인트 생성?!
# # checkpoint_filepath = '/root/spark/baseline/'
# fname = "checkpoint-{epoch:02d}-{val_loss:.2f}"
# checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min",
#     save_best_only=True, verbose=1)
# callbacks = [checkpoint]

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          500100    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 735,110
Trainable params: 735,110
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM, SimpleRNN
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D
nb_classes=2
vocab_sizes = 5001
neg_vocab_sizes = 5001
embedding_vector_length = 100
hidden_dims = 250
max_length = 100
dropout_ratio = 0.5
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_sizes, embedding_vector_length, input_length=max_length))

#어휘 크기(최대 정수 인덱스+1, 고밀도 임베딩의 치수, 입력 시퀀스의 길이

# 모델은 크기의 정수 행렬 (배치,
# input_length) 및 입력에서 가장 큰 정수 (즉, 단어 인덱스)
#은 999 (어휘 크기)보다 크지 않아야합니다.
# 이제 model.output_shape는 (None, 10, 64)이고, 여기서`None`은 배치입니다.
# dimension.
"""model.add(Conv1D(128, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(250, activation='relu'))"""

"""model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))"""

"""#model.add(Activation('relu'))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(256, 3,  padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
#model.add(Dense(128, activation='relu')) #
model.add(Dropout(dropout_ratio))
model.add(Activation('relu'))
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])"""


model.add(SimpleRNN(128,return_sequences=True))
model.add(SimpleRNN(128,return_sequences=True))
model.add(SimpleRNN(128))
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# #체크포인트 생성?!
# # checkpoint_filepath = '/root/spark/baseline/'
# fname = "checkpoint-{epoch:02d}-{val_loss:.2f}"
# checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min",
#     save_best_only=True, verbose=1)
# callbacks = [checkpoint]

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          500100    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100, 128)          29312     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 100, 128)          32896     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
_________________________________________________________________
activation (Activation)      (None, 2)                 0         
Total params: 595,462
Trainable params: 595,462
Non-trainable params: 0
__________________________________________________

In [18]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D
nb_classes=2
vocab_sizes = 5001
neg_vocab_sizes = 5001
embedding_vector_length = 100
hidden_dims = 250
max_length = 100
dropout_ratio = 0.5
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_sizes, embedding_vector_length, input_length=max_length))

#어휘 크기(최대 정수 인덱스+1, 고밀도 임베딩의 치수, 입력 시퀀스의 길이

# 모델은 크기의 정수 행렬 (배치,
# input_length) 및 입력에서 가장 큰 정수 (즉, 단어 인덱스)
#은 999 (어휘 크기)보다 크지 않아야합니다.
# 이제 model.output_shape는 (None, 10, 64)이고, 여기서`None`은 배치입니다.
# dimension.
model.add(Dropout(dropout_ratio))
model.add(Conv1D(256, 5, activation='relu'))
#model.add(Bidirectional(LSTM(128)))
model.add(GlobalMaxPooling1D())
model.add(Dense(250, activation='relu')) # 한 층 더 쌓을까..? relu 함수는 정류된 함수로 은닉층에서 많이 사용된다. -를 차단함!
# 은닉층으로 많이 사용되는 이유는 음수는 0으로 반환하므로 특정 양수값으로 수렴하지 않음. 그래서 기울기 소실이 발생하지 않음!
# 시그모이드 함수는 기울기 소실이 발생한다는 단점이 존재. 그래서 relu를 은닉층으로 쓰는 거임!
# 또한, 학습 속도가 매우 빠르다.
model.add(Dropout(dropout_ratio))
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
"""#model.add(Dropout(dropout_ratio))
model.add(Conv1D(256, 3,  padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
#model.add(Dense(128, activation='relu')) #
model.add(Dropout(dropout_ratio))
model.add(Activation('relu'))
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])"""

"""model.add(Bidirectional(LSTM(128)))
# model.add(GlobalMaxPooling1D())
model.add(Activation('relu')) # 한 층 더 쌓을까..? relu 함수는 정류된 함수로 은닉층에서 많이 사용된다. -를 차단함!
# 은닉층으로 많이 사용되는 이유는 음수는 0으로 반환하므로 특정 양수값으로 수렴하지 않음. 그래서 기울기 소실이 발생하지 않음!
# 시그모이드 함수는 기울기 소실이 발생한다는 단점이 존재. 그래서 relu를 은닉층으로 쓰는 거임!
# 또한, 학습 속도가 매우 빠르다."""
"""model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])"""

# #체크포인트 생성?!
# # checkpoint_filepath = '/root/spark/baseline/'
# fname = "checkpoint-{epoch:02d}-{val_loss:.2f}"
# checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min",
#     save_best_only=True, verbose=1)
# callbacks = [checkpoint]

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          500100    
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 96, 256)           128256    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               64250     
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 5

In [ ]:
######################fine tuning fourth window

In [30]:
from keras import optimizers, regularizers
from tensorflow.keras import optimizers

optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

In [28]:
help(ElephasEstimator)

NameError: name 'ElephasEstimator' is not defined

In [31]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("feature1")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(trans_model.to_json())
estimator.set_categorical_labels(True) # dense 1이면 False로 설정해야함
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(3)
estimator.set_epochs(10)
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("asynchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_c4622a31d51a

In [32]:
dl_pipeline = Pipeline(stages=[estimator])

In [26]:
trainqq.show(10)

+-----------+--------------------+--------------------+-----+
|label_index|            feature1|            feature2|label|
+-----------+--------------------+--------------------+-----+
|        1.0|[1.0,121.0,12.0,2...|[841.0,3042.0,413...|    1|
|        1.0|[2.0,924.0,1136.0...|[927.0,4860.0,416...|    1|
|        1.0|[2.0,3780.0,455.0...|[927.0,978.0,2633...|    1|
|        1.0|[3.0,2.0,355.0,15...|[927.0,1615.0,183...|    1|
|        1.0|[3.0,17.0,174.0,1...|[1710.0,1839.0,31...|    1|
|        1.0|[3.0,635.0,71.0,1...|[4043.0,4773.0,11...|    1|
|        1.0|[4.0,8.0,2.0,18.0...|[121.0,927.0,1371...|    1|
|        1.0|[4.0,8.0,2.0,18.0...|[121.0,927.0,1371...|    1|
|        1.0|[4.0,8.0,2.0,18.0...|[121.0,927.0,1371...|    1|
|        1.0|[4.0,8.0,2.0,18.0...|[121.0,927.0,1371...|    1|
+-----------+--------------------+--------------------+-----+
only showing top 10 rows



In [27]:
trainqq1

DataFrame[label_index: double, feature1: vector, feature2: vector, label: bigint]

In [ ]:
# pretr, prets

In [ ]:
import time
time1 = time.time()
my_dl1 = dl_pipeline.fit(trainqq) # lstm finetuning
time3 = time.time() - time1
print(time3)
aaaaaa = "완료!!"

>>> Fit model
 * Serving Flask app 'elephas.parameter.server' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.28.0.1:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


172.28.0.1 - - [29/Oct/2022 10:31:37] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:31:37] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:31:37] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:34:34] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:34:34] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:34:34] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:34:34] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:34:34] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:34:34] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:37:30] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:37:30] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:37:32] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:37:32] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2022 10:37:35] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [29/Oct/2

In [ ]:
91

In [37]:
time3

1257.723426580429

In [ ]:
import time
time1 = time.time()
my_dl1 = dl_pipeline.fit(trainqq1) # second winow
time3 = time.time() - time1
print(time3)
aaaaaa = "완료!!"

In [ ]:
import time
time1 = time.time()
my_dl1 = dl_pipeline.fit(trainqq1) # second winow
time3 = time.time() - time1
print(time3)
aaaaaa = "완료!!"

In [ ]:


from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

#pred_test = my_dl.transform(label_dft)
pred_test = my_dl1.transform(testqq)

from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType()) 
pred_test = pred_test.select(
    pred_test["label_index"], 
    pred_test["prediction"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)
pred_test.printSchema()
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l<0.5 else 0.0 , DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"],
    pred_test["prediction2"],
    list_to_vector_udf(pred_test["prediction2"]).alias("rawPrediction")
)


In [ ]:
pred = pred_test.toPandas()

In [26]:
spark

In [42]:
dfdf

,label_index,prediction2,rawPrediction
0,1.0,6.460475e-07,1.0
1,1.0,6.644039e-20,1.0
2,1.0,3.293934e-20,1.0
3,1.0,2.934441e-19,1.0
4,1.0,3.574041e-11,1.0
...,...,...,...
60323,0.0,2.997528e-11,0.0
60324,0.0,7.365455e-20,0.0
60325,0.0,1.002725e-20,0.0
60326,0.0,7.088632e-11,0.0


In [ ]:
pred_test.show()

In [ ]:
a123 = np.array(trainqq.select("feature1").collect())

In [ ]:
a1234 = np.array(trainqq.select("label_index").collect())

# test